In [1]:
import pandas as pd
import numpy as np

In [2]:
dataset = pd.read_csv(r'/Users/t_velpac/mission/WorkingCopy/Features_V3.csv')
dataset.drop(columns = ['Z_Score_HelpfulVotes','Z_Score_Words'], inplace=True)
dataset.rename(index=str, columns={"Helpful Votes": "Helpful_Votes"}, inplace=True)
dataset = dataset[dataset.Helpful_Votes != 0]


"""
Calculating Z-Scores again for Helpful Votes and Words
"""
dataset['Z_Score_Words'] = (dataset['Words'] - dataset['Words'].mean()) / dataset['Words'].std(ddof=0)
dataset['Z_Score_Helpful_Votes'] = (dataset['Helpful_Votes'] - dataset['Helpful_Votes'].mean()) / dataset['Helpful_Votes'].std(ddof=0)

"""
Dropping Helpful Votes and Words columns since we have their z-scores
"""
dataset.drop(columns = ["Helpful_Votes", "Words", "Date"], inplace=True)
dataset.head()

,Stars,Paragraphs,No.break tags,Percentage_Upper_Case,Percentage_Lower_Case,Avg_len_paragraph_per_review,Z_Score_Words,Z_Score_Helpful_Votes
0,4,8,14,3,90,166.125000,1.321147,0.550824
1,1,2,2,2,94,352.500000,0.255207,0.263646
2,1,1,0,3,90,556.000000,0.100722,0.263646
3,5,3,4,3,92,188.666667,0.100722,6.522436
4,1,1,0,3,96,216.000000,-0.447696,0.153843


In [3]:
percentage_of_0_votes = 100 - ((56911/200372)*100)
percentage_of_0_votes

71.5973289681193

In [4]:
"""Separating the independant variables from the dependant variable which is "Helpful Votes" in this case"""
X = dataset.iloc[:,0:-1].values
y = dataset.iloc[:,-1].values

In [5]:
"""Splitting the data into training data and testing data"""

from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y, test_size=0.2,random_state=1)

"""Scaling the data using StandardScaler from sklearn package"""
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.fit_transform(X_test)

In [6]:
# importing keras and other required functions
import keras
from keras.models import Sequential
from keras.layers import Dense

Using TensorFlow backend.


In [7]:
""" Building a 3 layer ANN with high number of nodes """

regressor = Sequential()
regressor.add(Dense(100, kernel_initializer = 'normal',activation = 'relu',input_dim = 7))
regressor.add(Dense(100, kernel_initializer = 'normal', activation = 'relu'))
regressor.add(Dense(1, kernel_initializer = 'normal'))

""" Compiling the regressor """
regressor.compile(optimizer = 'adam', loss = 'mean_squared_error')

""" Fitting the Artifilial Neural Network to our training data """
regressor.fit(X_train, y_train, batch_size=5, epochs=100, verbose = 1)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Epoch 1/100
45528/45528 [==============================] - 8s 183us/step - loss: 0.9730
Epoch 2/100
45528/45528 [==============================] - 7s 164us/step - loss: 0.9738
Epoch 3/100
45528/45528 [==============================] - 8s 175us/step - loss: 0.9728
Epoch 4/100
45528/45528 [==============================] - 8s 168us/step - loss: 0.9721
Epoch 5/100
45528/45528 [==============================] - 8s 185us/step - loss: 0.9717
Epoch 6/100
45528/45528 [==============================] - 8s 185us/step - loss: 0.9724
Epoch 7/100
45528/45528 [==============================] - 7s 164us/step - loss: 0.9724
Epoch 8/100
45528/45528 [==============================] - 8s 186us/step - loss: 0.9725
Epoch 9/100
45528/45528 [==============================] - 9s 204us/step - loss: 0.9715
Epoch 10/100
45528/45528 [==============================] - 9s 193us/step - loss: 0.9712

In [8]:
""" Predicting the values for Helpful Votes on the test data """
y_pred = regressor.predict(X_test)

""" Creating a dataframe to compare the actual values against predicted values """
y_pred = y_pred.reshape(11383,)
temp = {'Actual Values': y_test,'Predicted Values': y_pred}
y_compare = pd.DataFrame(temp)

""" Calculating the Mean Squared Error to estimate the efficiency of the ANN"""
# We are calculating this MSE in two steps. Don't get confused.
y_compare['Mean Squared Error'] = (np.diff(y_compare.values) ** 2)
y_compare['Mean Squared Error'] = np.mean(y_compare['Mean Squared Error'])

# Now calculating the Root Mean Squared Error(RMSE)
y_compare['Root Mean Squared Error'] = y_compare['Mean Squared Error']**0.5

# Calculating Mean Absolute Error
y_compare['Mean Abs. Error'] = np.mean(abs(y_compare['Actual Values'] - y_compare['Predicted Values']))

y_compare

,Actual Values,Predicted Values,Mean Squared Error,Root Mean Squared Error,Mean Abs. Error
0,-0.057317,-0.016805,1.103208,1.050337,0.088073
1,-0.006639,-0.008149,1.103208,1.050337,0.088073
2,-0.057317,-0.063594,1.103208,1.050337,0.088073
3,-0.057317,-0.028262,1.103208,1.050337,0.088073
4,-0.057317,-0.044178,1.103208,1.050337,0.088073
5,-0.053094,-0.055906,1.103208,1.050337,0.088073
6,-0.044648,-0.024733,1.103208,1.050337,0.088073
7,-0.057317,0.061495,1.103208,1.050337,0.088073
8,-0.057317,-0.026754,1.103208,1.050337,0.088073
9,-0.040425,0.005791,1.103208,1.050337,0.088073


In [9]:
""" Now we will also predict the y values on the training set just to calculate MSE and RMSE """

y_pred_train = regressor.predict(X_train)

""" Creating a dataframe to compare the actual values against the predicted values for the training set """
y_pred_train = y_pred_train.reshape(45528,)
temp_train = {'Actual Values(Training)':y_train, 'Predicted Values(Training)': y_pred_train }
y_compare_train = pd.DataFrame(temp_train)

""" Calculating the Mean Squared Error to estimate the efficiency of the ANN on TRAINING SET"""
# We are calculating this MSE in two steps. Don't get confused.
y_compare_train['Mean Squared Error'] = (np.diff(y_compare_train.values) ** 2)
y_compare_train['Mean Squared Error'] = np.mean(y_compare_train['Mean Squared Error'])

# Now calculating the Root Mean Squared Error(RMSE)
y_compare_train['Root Mean Squared Error'] = y_compare_train['Mean Squared Error']**0.5

# Calculating Mean Absolute Error
y_compare_train['Mean Abs. Error'] = np.mean(abs(y_compare_train['Actual Values(Training)'] - y_compare_train['Predicted Values(Training)']))

y_compare_train

,Actual Values(Training),Predicted Values(Training),Mean Squared Error,Root Mean Squared Error,Mean Abs. Error
0,-0.057317,-0.021849,0.970964,0.985375,0.086112
1,-0.057317,-0.013909,0.970964,0.985375,0.086112
2,-0.053094,-0.003715,0.970964,0.985375,0.086112
3,-0.057317,-0.031203,0.970964,0.985375,0.086112
4,-0.057317,-0.032519,0.970964,0.985375,0.086112
5,-0.010862,-0.034904,0.970964,0.985375,0.086112
6,-0.040425,-0.002829,0.970964,0.985375,0.086112
7,12.472933,0.068633,0.970964,0.985375,0.086112
8,-0.023532,-0.033891,0.970964,0.985375,0.086112
9,-0.057317,-0.030872,0.970964,0.985375,0.086112


In [10]:
"""


Now we will be training a different ANN with lower number of nodes

7-4-4-1

"""

'\n\n\nNow we will be training a different ANN with lower number of nodes\n\n7-4-4-1\n\n'

In [11]:
""" Building a 3 layer ANN with less number of nodes """

regressor_1 = Sequential()
regressor_1.add(Dense(4, kernel_initializer = 'normal',activation = 'relu',input_dim = 7))
regressor_1.add(Dense(4, kernel_initializer = 'normal', activation = 'relu'))
regressor_1.add(Dense(1, kernel_initializer = 'normal'))

""" Compiling the regressor """
regressor_1.compile(optimizer = 'adam', loss = 'mean_squared_error')

""" Fitting the Artifilial Neural Network to our training data """
regressor_1.fit(X_train, y_train, batch_size=5, epochs=100, verbose = 1)

Epoch 1/100
45528/45528 [==============================] - 8s 171us/step - loss: 0.9732
Epoch 2/100
45528/45528 [==============================] - 7s 164us/step - loss: 0.9715
Epoch 3/100
45528/45528 [==============================] - 7s 164us/step - loss: 0.9710
Epoch 4/100
45528/45528 [==============================] - 8s 166us/step - loss: 0.9708
Epoch 5/100
45528/45528 [==============================] - 8s 175us/step - loss: 0.9708
Epoch 6/100
45528/45528 [==============================] - 7s 162us/step - loss: 0.9701
Epoch 7/100
45528/45528 [==============================] - 8s 165us/step - loss: 0.9705
Epoch 8/100
45528/45528 [==============================] - 8s 167us/step - loss: 0.9702
Epoch 9/100
45528/45528 [==============================] - 7s 160us/step - loss: 0.9702
Epoch 10/100
45528/45528 [==============================] - 7s 161us/step - loss: 0.9705
Epoch 11/100
45528/45528 [==============================] - 8s 166us/step - loss: 0.9700
Epoch 12/100
45528/45528 [====

45528/45528 [==============================] - 8s 168us/step - loss: 0.9688
Epoch 94/100
45528/45528 [==============================] - 7s 164us/step - loss: 0.9687
Epoch 95/100
45528/45528 [==============================] - 7s 163us/step - loss: 0.9687
Epoch 96/100
45528/45528 [==============================] - 7s 163us/step - loss: 0.9688
Epoch 97/100
45528/45528 [==============================] - 8s 176us/step - loss: 0.9685
Epoch 98/100
45528/45528 [==============================] - 8s 165us/step - loss: 0.9685
Epoch 99/100
45528/45528 [==============================] - 8s 166us/step - loss: 0.9683
Epoch 100/100
45528/45528 [==============================] - 7s 164us/step - loss: 0.9691


In [12]:
""" Predicting the values for Helpful Votes on the test data """
y_pred_1 = regressor_1.predict(X_test)

""" Creating a dataframe to compare the actual values against predicted values """
y_pred_1 = y_pred_1.reshape(11383,)
temp = {'Actual Values': y_test,'Predicted Values': y_pred_1}
y_compare_1 = pd.DataFrame(temp)

""" Calculating the Mean Squared Error to estimate the efficiency of the ANN"""
# We are calculating this MSE in two steps. Don't get confused.
y_compare_1['Mean Squared Error'] = (np.diff(y_compare_1.values) ** 2)
y_compare_1['Mean Squared Error'] = np.mean(y_compare_1['Mean Squared Error'])

# Now calculating the Root Mean Squared Error(RMSE)
y_compare_1['Root Mean Squared Error'] = y_compare_1['Mean Squared Error']**0.5

# Calculating Mean Absolute Error
y_compare_1['Mean Abs. Error'] = np.mean(abs(y_compare_1['Actual Values'] - y_compare_1['Predicted Values']))

y_compare_1

,Actual Values,Predicted Values,Mean Squared Error,Root Mean Squared Error,Mean Abs. Error
0,-0.057317,-0.035972,1.104595,1.050997,0.08526
1,-0.006639,-0.017618,1.104595,1.050997,0.08526
2,-0.057317,-0.057766,1.104595,1.050997,0.08526
3,-0.057317,-0.045323,1.104595,1.050997,0.08526
4,-0.057317,-0.057173,1.104595,1.050997,0.08526
5,-0.053094,-0.040844,1.104595,1.050997,0.08526
6,-0.044648,-0.047207,1.104595,1.050997,0.08526
7,-0.057317,0.131161,1.104595,1.050997,0.08526
8,-0.057317,-0.043006,1.104595,1.050997,0.08526
9,-0.040425,-0.013965,1.104595,1.050997,0.08526


In [13]:
""" Now we will also predict the y values on the training set just to calculate MSE and RMSE """

y_pred_train_1 = regressor_1.predict(X_train)

""" Creating a dataframe to compare the actual values against the predicted values for the training set """
y_pred_train_1 = y_pred_train_1.reshape(45528,)
temp_train = {'Actual Values(Training)':y_train, 'Predicted Values(Training)': y_pred_train_1 }
y_compare_train_1 = pd.DataFrame(temp_train)

""" Calculating the Mean Squared Error to estimate the efficiency of the ANN on TRAINING SET"""
# We are calculating this MSE in two steps. Don't get confused.
y_compare_train_1['Mean Squared Error'] = (np.diff(y_compare_train_1.values) ** 2)
y_compare_train_1['Mean Squared Error'] = np.mean(y_compare_train_1['Mean Squared Error'])

# Now calculating the Root Mean Squared Error(RMSE)
y_compare_train_1['Root Mean Squared Error'] = y_compare_train_1['Mean Squared Error']**0.5

# Calculating Mean Absolute Error
y_compare_train_1['Mean Abs. Error'] = np.mean(abs(y_compare_train_1['Actual Values(Training)'] - y_compare_train_1['Predicted Values(Training)']))

y_compare_train_1

,Actual Values(Training),Predicted Values(Training),Mean Squared Error,Root Mean Squared Error,Mean Abs. Error
0,-0.057317,-0.041254,0.966932,0.983327,0.083103
1,-0.057317,-0.036322,0.966932,0.983327,0.083103
2,-0.053094,-0.013965,0.966932,0.983327,0.083103
3,-0.057317,-0.042682,0.966932,0.983327,0.083103
4,-0.057317,-0.042292,0.966932,0.983327,0.083103
5,-0.010862,-0.041071,0.966932,0.983327,0.083103
6,-0.040425,-0.013965,0.966932,0.983327,0.083103
7,12.472933,-0.013965,0.966932,0.983327,0.083103
8,-0.023532,-0.037772,0.966932,0.983327,0.083103
9,-0.057317,-0.044908,0.966932,0.983327,0.083103


In [14]:
"""


Now, we will train an ANN with a very high number of nodes. And comparitively more layers.

7-50-100-200-100-50-20-1
"""

'\n\n\nNow, we will train an ANN with a very high number of nodes. And comparitively more layers.\n\n7-50-100-200-100-50-20-1\n'

In [15]:
""" Building a deep and wide ANN with high number of nodes """

from keras.layers.advanced_activations import LeakyReLU
regressor_2 = Sequential()
regressor_2.add(Dense(50, kernel_initializer = 'normal',input_dim = 7))
regressor_2.add(LeakyReLU(alpha=0.05))
regressor_2.add(Dense(100, kernel_initializer = 'normal'))
regressor_2.add(LeakyReLU(alpha=0.05))
regressor_2.add(Dense(200, kernel_initializer = 'normal'))
regressor_2.add(LeakyReLU(alpha=0.05))
regressor_2.add(Dense(100, kernel_initializer = 'normal'))
regressor_2.add(LeakyReLU(alpha=0.05))
regressor_2.add(Dense(50, kernel_initializer = 'normal'))
regressor_2.add(LeakyReLU(alpha=0.05))
regressor_2.add(Dense(20, kernel_initializer = 'normal'))
regressor_2.add(LeakyReLU(alpha=0.05))
regressor_2.add(Dense(1, kernel_initializer = 'normal'))

""" Compiling the regressor """
regressor_2.compile(optimizer = 'adam', loss = 'mean_squared_error')

""" Fitting the Artifilial Neural Network to our training data """
regressor_2.fit(X_train, y_train, batch_size=5, epochs=100, verbose = 1)

Epoch 1/100
45528/45528 [==============================] - 14s 307us/step - loss: 0.9740
Epoch 2/100
45528/45528 [==============================] - 13s 288us/step - loss: 0.9742
Epoch 3/100
45528/45528 [==============================] - 12s 259us/step - loss: 0.9736
Epoch 4/100
45528/45528 [==============================] - 12s 255us/step - loss: 0.9734
Epoch 5/100
45528/45528 [==============================] - 12s 257us/step - loss: 0.9728
Epoch 6/100
45528/45528 [==============================] - 12s 256us/step - loss: 0.9738
Epoch 7/100
45528/45528 [==============================] - 12s 254us/step - loss: 0.9737
Epoch 8/100
45528/45528 [==============================] - 12s 253us/step - loss: 0.9724
Epoch 9/100
45528/45528 [==============================] - 12s 255us/step - loss: 0.9734
Epoch 10/100
45528/45528 [==============================] - 12s 253us/step - loss: 0.9723
Epoch 11/100
45528/45528 [==============================] - 12s 254us/step - loss: 0.9755
Epoch 12/100
45528/

In [16]:
""" Predicting the values for Helpful Votes on the test data """
y_pred_2 = regressor_2.predict(X_test)

""" Creating a dataframe to compare the actual values against predicted values """
y_pred_2 = y_pred_2.reshape(11383,)
temp = {'Actual Values': y_test,'Predicted Values': y_pred_2}
y_compare_2 = pd.DataFrame(temp)

""" Calculating the Mean Squared Error to estimate the efficiency of the ANN"""
# We are calculating this MSE in two steps. Don't get confused.
y_compare_2['Mean Squared Error'] = (np.diff(y_compare_2.values) ** 2)
y_compare_2['Mean Squared Error'] = np.mean(y_compare_2['Mean Squared Error'])

# Now calculating the Root Mean Squared Error(RMSE)
y_compare_2['Root Mean Squared Error'] = y_compare_2['Mean Squared Error']**0.5

# Calculating Mean Absolute Error
y_compare_2['Mean Abs. Error'] = np.mean(abs(y_compare_2['Actual Values'] - y_compare_2['Predicted Values']))

y_compare_2

,Actual Values,Predicted Values,Mean Squared Error,Root Mean Squared Error,Mean Abs. Error
0,-0.057317,-0.050163,1.101877,1.049703,0.08661
1,-0.006639,0.007466,1.101877,1.049703,0.08661
2,-0.057317,-0.060926,1.101877,1.049703,0.08661
3,-0.057317,-0.050722,1.101877,1.049703,0.08661
4,-0.057317,-0.054982,1.101877,1.049703,0.08661
5,-0.053094,-0.023887,1.101877,1.049703,0.08661
6,-0.044648,-0.058425,1.101877,1.049703,0.08661
7,-0.057317,0.082729,1.101877,1.049703,0.08661
8,-0.057317,-0.050198,1.101877,1.049703,0.08661
9,-0.040425,0.023992,1.101877,1.049703,0.08661


In [17]:
""" Now we will also predict the y values on the training set just to calculate MSE and RMSE """

y_pred_train_2 = regressor_2.predict(X_train)

""" Creating a dataframe to compare the actual values against the predicted values for the training set """
y_pred_train_2 = y_pred_train_2.reshape(45528,)
temp_train = {'Actual Values(Training)':y_train, 'Predicted Values(Training)': y_pred_train_2 }
y_compare_train_2 = pd.DataFrame(temp_train)

""" Calculating the Mean Squared Error to estimate the efficiency of the ANN on TRAINING SET"""
# We are calculating this MSE in two steps. Don't get confused.
y_compare_train_2['Mean Squared Error'] = (np.diff(y_compare_train_2.values) ** 2)
y_compare_train_2['Mean Squared Error'] = np.mean(y_compare_train_2['Mean Squared Error'])

# Now calculating the Root Mean Squared Error(RMSE)
y_compare_train_2['Root Mean Squared Error'] = y_compare_train_2['Mean Squared Error']**0.5

# Calculating Mean Absolute Error
y_compare_train_2['Mean Abs. Error'] = np.mean(abs(y_compare_train_2['Actual Values(Training)'] - y_compare_train_2['Predicted Values(Training)']))

y_compare_train_2

,Actual Values(Training),Predicted Values(Training),Mean Squared Error,Root Mean Squared Error,Mean Abs. Error
0,-0.057317,-0.054343,0.96956,0.984663,0.084572
1,-0.057317,-0.046971,0.96956,0.984663,0.084572
2,-0.053094,0.003423,0.96956,0.984663,0.084572
3,-0.057317,-0.051991,0.96956,0.984663,0.084572
4,-0.057317,-0.048740,0.96956,0.984663,0.084572
5,-0.010862,-0.011619,0.96956,0.984663,0.084572
6,-0.040425,-0.006374,0.96956,0.984663,0.084572
7,12.472933,0.067522,0.96956,0.984663,0.084572
8,-0.023532,-0.048859,0.96956,0.984663,0.084572
9,-0.057317,-0.053817,0.96956,0.984663,0.084572


In [18]:
""" Building a 3 layer ANN with 2 layers of 50 nodes """

regressor_3 = Sequential()
regressor_3.add(Dense(50, kernel_initializer = 'normal',activation = 'relu',input_dim = 7))
regressor_3.add(Dense(50, kernel_initializer = 'normal', activation = 'relu'))
regressor_3.add(Dense(1, kernel_initializer = 'normal'))

""" Compiling the regressor """
regressor_3.compile(optimizer = 'adam', loss = 'mean_squared_error')

""" Fitting the Artifilial Neural Network to our training data """
regressor_3.fit(X_train, y_train, batch_size=5, epochs=100, verbose = 1)

Epoch 1/100
45528/45528 [==============================] - 8s 173us/step - loss: 0.9724
Epoch 2/100
45528/45528 [==============================] - 8s 166us/step - loss: 0.9714
Epoch 3/100
45528/45528 [==============================] - 8s 166us/step - loss: 0.9709
Epoch 4/100
45528/45528 [==============================] - 8s 165us/step - loss: 0.9720
Epoch 5/100
45528/45528 [==============================] - 8s 166us/step - loss: 0.9716
Epoch 6/100
45528/45528 [==============================] - 8s 165us/step - loss: 0.9722
Epoch 7/100
45528/45528 [==============================] - 8s 166us/step - loss: 0.9708
Epoch 8/100
45528/45528 [==============================] - 8s 170us/step - loss: 0.9706
Epoch 9/100
45528/45528 [==============================] - 8s 177us/step - loss: 0.9709
Epoch 10/100
45528/45528 [==============================] - 8s 167us/step - loss: 0.9707
Epoch 11/100
45528/45528 [==============================] - 8s 166us/step - loss: 0.9708
Epoch 12/100
45528/45528 [====

In [19]:
""" Predicting the values for Helpful Votes on the test data """
y_pred_3 = regressor_3.predict(X_test)

""" Creating a dataframe to compare the actual values against predicted values """
y_pred_3 = y_pred_3.reshape(11383,)
temp = {'Actual Values': y_test,'Predicted Values': y_pred_3}
y_compare_3 = pd.DataFrame(temp)

""" Calculating the Mean Squared Error to estimate the efficiency of the ANN"""
# We are calculating this MSE in two steps. Don't get confused.
y_compare_3['Mean Squared Error'] = (np.diff(y_compare_3.values) ** 2)
y_compare_3['Mean Squared Error'] = np.mean(y_compare_3['Mean Squared Error'])

# Now calculating the Root Mean Squared Error(RMSE)
y_compare_3['Root Mean Squared Error'] = y_compare_3['Mean Squared Error']**0.5

# Calculating Mean Absolute Error
y_compare_3['Mean Abs. Error'] = np.mean(abs(y_compare_3['Actual Values'] - y_compare_3['Predicted Values']))

y_compare_3

,Actual Values,Predicted Values,Mean Squared Error,Root Mean Squared Error,Mean Abs. Error
0,-0.057317,-0.005332,1.10514,1.051256,0.125425
1,-0.006639,0.019325,1.10514,1.051256,0.125425
2,-0.057317,-0.032504,1.10514,1.051256,0.125425
3,-0.057317,-0.005214,1.10514,1.051256,0.125425
4,-0.057317,-0.014914,1.10514,1.051256,0.125425
5,-0.053094,-0.028529,1.10514,1.051256,0.125425
6,-0.044648,-0.017451,1.10514,1.051256,0.125425
7,-0.057317,0.212358,1.10514,1.051256,0.125425
8,-0.057317,0.001506,1.10514,1.051256,0.125425
9,-0.040425,0.013820,1.10514,1.051256,0.125425


In [20]:
""" Now we will also predict the y values on the training set just to calculate MSE and RMSE """

y_pred_train_3 = regressor_3.predict(X_train)

""" Creating a dataframe to compare the actual values against the predicted values for the training set """
y_pred_train_3 = y_pred_train_3.reshape(45528,)
temp_train = {'Actual Values(Training)':y_train, 'Predicted Values(Training)': y_pred_train_3 }
y_compare_train_3 = pd.DataFrame(temp_train)

""" Calculating the Mean Squared Error to estimate the efficiency of the ANN on TRAINING SET"""
# We are calculating this MSE in two steps. Don't get confused.
y_compare_train_3['Mean Squared Error'] = (np.diff(y_compare_train_3.values) ** 2)
y_compare_train_3['Mean Squared Error'] = np.mean(y_compare_train_3['Mean Squared Error'])

# Now calculating the Root Mean Squared Error(RMSE)
y_compare_train_3['Root Mean Squared Error'] = y_compare_train_3['Mean Squared Error']**0.5

# Calculating Mean Absolute Error
y_compare_train_3['Mean Abs. Error'] = np.mean(abs(y_compare_train_3['Actual Values(Training)'] - y_compare_train_3['Predicted Values(Training)']))

y_compare_train_3

,Actual Values(Training),Predicted Values(Training),Mean Squared Error,Root Mean Squared Error,Mean Abs. Error
0,-0.057317,0.013650,0.968417,0.984082,0.123561
1,-0.057317,0.010785,0.968417,0.984082,0.123561
2,-0.053094,0.012358,0.968417,0.984082,0.123561
3,-0.057317,-0.006127,0.968417,0.984082,0.123561
4,-0.057317,-0.002343,0.968417,0.984082,0.123561
5,-0.010862,-0.013716,0.968417,0.984082,0.123561
6,-0.040425,0.015456,0.968417,0.984082,0.123561
7,12.472933,0.065269,0.968417,0.984082,0.123561
8,-0.023532,-0.006171,0.968417,0.984082,0.123561
9,-0.057317,-0.016575,0.968417,0.984082,0.123561


In [21]:
""" Building a 3 layer ANN with 2 layers of 20 nodes """

regressor_4 = Sequential()
regressor_4.add(Dense(20, kernel_initializer = 'normal',activation = 'relu',input_dim = 7))
regressor_4.add(Dense(20, kernel_initializer = 'normal', activation = 'relu'))
regressor_4.add(Dense(1, kernel_initializer = 'normal'))

""" Compiling the regressor """
regressor_4.compile(optimizer = 'adam', loss = 'mean_squared_error')

""" Fitting the Artifilial Neural Network to our training data """
regressor_4.fit(X_train, y_train, batch_size=5, epochs=100, verbose = 1)

Epoch 1/100
45528/45528 [==============================] - 5788s 127ms/step - loss: 0.9720
Epoch 2/100
45528/45528 [==============================] - 126s 3ms/step - loss: 0.9713
Epoch 3/100
45528/45528 [==============================] - 9s 194us/step - loss: 0.9718
Epoch 4/100
45528/45528 [==============================] - 10s 220us/step - loss: 0.9711
Epoch 5/100
45528/45528 [==============================] - 8s 178us/step - loss: 0.9709
Epoch 6/100
45528/45528 [==============================] - 8s 177us/step - loss: 0.9706
Epoch 7/100
45528/45528 [==============================] - 9s 194us/step - loss: 0.9707
Epoch 8/100
45528/45528 [==============================] - 9s 189us/step - loss: 0.9712
Epoch 9/100
45528/45528 [==============================] - 8s 181us/step - loss: 0.9706
Epoch 10/100
45528/45528 [==============================] - 8s 178us/step - loss: 0.9706
Epoch 11/100
45528/45528 [==============================] - 8s 172us/step - loss: 0.9708
Epoch 12/100
45528/45528 [

In [22]:
""" Predicting the values for Helpful Votes on the test data """
y_pred_4 = regressor_4.predict(X_test)

""" Creating a dataframe to compare the actual values against predicted values """
y_pred_4 = y_pred_4.reshape(11383,)
temp = {'Actual Values': y_test,'Predicted Values': y_pred_4}
y_compare_4 = pd.DataFrame(temp)

""" Calculating the Mean Squared Error to estimate the efficiency of the ANN"""
# We are calculating this MSE in two steps. Don't get confused.
y_compare_4['Mean Squared Error'] = (np.diff(y_compare_4.values) ** 2)
y_compare_4['Mean Squared Error'] = np.mean(y_compare_4['Mean Squared Error'])

# Now calculating the Root Mean Squared Error(RMSE)
y_compare_4['Root Mean Squared Error'] = y_compare_4['Mean Squared Error']**0.5

# Calculating Mean Absolute Error
y_compare_4['Mean Abs. Error'] = np.mean(abs(y_compare_4['Actual Values'] - y_compare_4['Predicted Values']))

y_compare_4

,Actual Values,Predicted Values,Mean Squared Error,Root Mean Squared Error,Mean Abs. Error
0,-0.057317,-0.048850,1.103135,1.050302,0.081205
1,-0.006639,-0.015007,1.103135,1.050302,0.081205
2,-0.057317,-0.065118,1.103135,1.050302,0.081205
3,-0.057317,-0.054219,1.103135,1.050302,0.081205
4,-0.057317,-0.064231,1.103135,1.050302,0.081205
5,-0.053094,-0.046557,1.103135,1.050302,0.081205
6,-0.044648,-0.056977,1.103135,1.050302,0.081205
7,-0.057317,0.110021,1.103135,1.050302,0.081205
8,-0.057317,-0.051805,1.103135,1.050302,0.081205
9,-0.040425,0.013746,1.103135,1.050302,0.081205


In [23]:
""" Now we will also predict the y values on the training set just to calculate MSE and RMSE """

y_pred_train_4 = regressor_4.predict(X_train)

""" Creating a dataframe to compare the actual values against the predicted values for the training set """
y_pred_train_4 = y_pred_train_4.reshape(45528,)
temp_train = {'Actual Values(Training)':y_train, 'Predicted Values(Training)': y_pred_train_4 }
y_compare_train_4 = pd.DataFrame(temp_train)

""" Calculating the Mean Squared Error to estimate the efficiency of the ANN on TRAINING SET"""
# We are calculating this MSE in two steps. Don't get confused.
y_compare_train_4['Mean Squared Error'] = (np.diff(y_compare_train_4.values) ** 2)
y_compare_train_4['Mean Squared Error'] = np.mean(y_compare_train_4['Mean Squared Error'])

# Now calculating the Root Mean Squared Error(RMSE)
y_compare_train_4['Root Mean Squared Error'] = y_compare_train_4['Mean Squared Error']**0.5

# Calculating Mean Absolute Error
y_compare_train_4['Mean Abs. Error'] = np.mean(abs(y_compare_train_4['Actual Values(Training)'] - y_compare_train_4['Predicted Values(Training)']))

y_compare_train_4

,Actual Values(Training),Predicted Values(Training),Mean Squared Error,Root Mean Squared Error,Mean Abs. Error
0,-0.057317,-0.053119,0.968675,0.984213,0.079159
1,-0.057317,-0.046575,0.968675,0.984213,0.079159
2,-0.053094,-0.000166,0.968675,0.984213,0.079159
3,-0.057317,-0.052742,0.968675,0.984213,0.079159
4,-0.057317,-0.052691,0.968675,0.984213,0.079159
5,-0.010862,-0.045613,0.968675,0.984213,0.079159
6,-0.040425,-0.018176,0.968675,0.984213,0.079159
7,12.472933,0.037346,0.968675,0.984213,0.079159
8,-0.023532,-0.048815,0.968675,0.984213,0.079159
9,-0.057317,-0.054968,0.968675,0.984213,0.079159
